# HydroWaterFactorModel model tutorial

> **Set up**
>
> To run this notebook, first install the Julia kernel for Jupyter Notebooks using [IJulia](https://julialang.github.io/IJulia.jl/stable/manual/installation/), then [create an environment](https://pkgdocs.julialang.org/v1/environments/) for this tutorial with the packages listed with `using <PackageName>` further down.
>
> This tutorial has demonstrated compatibility with these package versions. If you run into any errors, first check your package versions for consistency using `Pkg.status()`.
>
 > ```
 > Status `~/work/HydroPowerSimulations.jl/HydroPowerSimulations.jl/docs/Project.toml`
 >   [a93c6f00] DataFrames v1.8.1
 >   [864edb3b] DataStructures v0.19.3
 >   [e30172f5] Documenter v1.16.1
 >   [d12716ef] DocumenterInterLinks v1.1.0
 >   [35a29f4d] DocumenterTools v0.1.21
 >   [87dc4568] HiGHS v1.21.0
 >   [fc1677e0] HydroPowerSimulations v0.13.1 `~/work/HydroPowerSimulations.jl/HydroPowerSimulations.jl`
 >   [2cd47ed4] InfrastructureSystems v3.3.0
 >   [b6b21f68] Ipopt v1.14.0
 >   [23fbe1c1] Latexify v0.16.10
 >   [98b081ad] Literate v2.21.0
 > ⌅ [94fada2c] PowerFlows v0.13.1
 >   [e690365d] PowerSimulations v0.32.4
 >   [f00506e0] PowerSystemCaseBuilder v2.2.0
 >   [bcd98974] PowerSystems v5.4.0
 >   [08abe8d2] PrettyTables v3.1.2
 >   [9e3dc215] TimeSeries v0.25.2
 > Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`
 > 
 > ```


> *Note*
>
>
> `HydroPowerSimulations.jl` is an extension library of [`PowerSimulations.jl`](https://nrel-sienna.github.io/PowerSimulations.jl/latest/) for modeling hydro units. Users are encouraged to review the [single-step tutorial in `PowerSimulations.jl`](https://nrel-sienna.github.io/PowerSimulations.jl/latest/tutorials/decision_problem/) before this tutorial.
>
## Load packages

In [ ]:
using PowerSystems
using PowerSimulations
using HydroPowerSimulations
using PowerSystemCaseBuilder
using Ipopt ## solver

## Data

> *Note*
>
>
> `PowerSystemCaseBuilder.jl` is a helper library that makes it easier to reproduce examples in the documentation and tutorials. Normally you would pass your local files to create the system data instead of calling the function `build_system`.
> For more details visit [PowerSystemCaseBuilder README](https://github.com/NREL-Sienna/PowerSystemCaseBuilder.jl/blob/main/README.md)

In [ ]:
sys = build_system(PSITestSystems, "c_sys5_hy_turbine_energy")

With a single `PowerSystems.HydroTurbine` connected downstream to a `PowerSystems.HydroReservoir`:

In [ ]:
reservoir = only(get_components(HydroReservoir, sys))

Set the storage level limits using `set_storage_level_limits!`. Here the limits are set between $4000 and 6000 m^3$ .

In [ ]:
set_storage_level_limits!(reservoir, (min = 4000, max = 6000))

Set the lower bound of reservoir volume using `set_level_targets!`. Here the level target is set at $0.9 \cdot 6000 = 5400 m^3$.

In [ ]:
set_level_targets!(reservoir, 0.9)

Set the head_to_volume factor to 1.0 using `set_head_to_volume_factor!`, since is an energy model and no conversion is needed.

In [ ]:
set_head_to_volume_factor!(reservoir, LinearCurve(1.0))

## Decision Model

Setting up the formulations based on [`PowerSimulations.jl`](https://nrel-sienna.github.io/PowerSimulations.jl/latest/formulation_library/Introduction/):

In [ ]:
template = ProblemTemplate(CopperPlatePowerModel)
set_device_model!(template, ThermalStandard, ThermalBasicDispatch)
set_device_model!(template, PowerLoad, StaticPowerLoad)

but, now we also include the HydroReservoir and HydroTurbine using `HydroWaterFactorModel`:

In [ ]:
set_device_model!(template, HydroReservoir, HydroWaterFactorModel)
set_device_model!(template, HydroTurbine, HydroWaterFactorModel)

With the template properly set-up, we construct, build and solve the optimization problem:

In [ ]:
model = DecisionModel(template, sys; optimizer = Ipopt.Optimizer)
build!(model; output_dir = mktempdir())
solve!(model)

## Exploring Results

Results can be explored using:

In [ ]:
results = OptimizationProblemResults(model)

Use `read_variable` to read in the dispatch variable results for the hydro:

In [ ]:
power =
    read_variable(
        results,
        "ActivePowerVariable__HydroTurbine";
        table_format = TableFormat.WIDE,
    )

or the volume level of the reservoir

In [ ]:
volume =
    read_variable(
        results,
        "HydroReservoirVolumeVariable__HydroReservoir";
        table_format = TableFormat.WIDE,
    )

Note that the final reservoir level is between the set level target and the maximum storage level.